<a href="https://colab.research.google.com/github/creyesvera/Deep_Q_learning/blob/main/DeepLearning_Creaci%C3%B3n_de_df_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:

#from google.colab import drive
#drive.mount('/content/drive')


```markdown
To access files from GitHub, you can clone a repository using the `!git clone` command. Replace `YOUR_GITHUB_REPOSITORY_URL` with the URL of the repository you want to clone.
```

In [36]:
# Example: Clone a GitHub repository
!git clone https://github.com/creyesvera/Deep_Q_learning.git

# After cloning, you can list the contents of the cloned repository
# For example, if the repository is named 'REPOSITORY', you can do:
# !ls REPOSITORY

fatal: destination path 'Deep_Q_learning' already exists and is not an empty directory.


In [37]:
## Visualizar datos
import pandas as pd
import os

## Graficar datos
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

## Discretización
import numpy as np

## Q_learning
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Tuple
from tabulate import tabulate

#Cargando datos sensados

In [38]:
file_path_horario = '/content/Deep_Q_learning/data/Horario de clases.xlsx'
df_horario = pd.read_excel(file_path_horario)
df_horario.head()

,Hora,Lunes,Martes,Miércoles,Jueves,Viernes
0,7:00-7:30,NaN,NaN,NaN,NaN,NaN
1,7:30-8:00,TLMG1033 Redes Inalámbricas y de Sensores (Nés...,Laboratorio abierto,Laboratorio abierto,NaN,NaN
2,8:00-8:30,TLMG1033 Redes Inalámbricas y de Sensores (Nés...,Laboratorio abierto,Laboratorio abierto,NaN,NaN
3,8:30-9:00,TLMG1033 Redes Inalámbricas y de Sensores (Nés...,Laboratorio abierto,Laboratorio abierto,NaN,NaN
4,9:00-9:30,TLMG1037 Programación de Sistemas Telemáticos ...,TLMG1037 Programación de Sistemas Telemáticos ...,Laboratorio abierto,Laboratorio abierto,TLMG1023 Ambientes Inteligentes (Néstor Arreaga)


In [39]:
df_horario = df_horario.copy()
days = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']

for day in days:
    # Replace values that are not null and not 'Laboratorio abierto' with 'Clase'
    mask = (df_horario[day].notna()) & (df_horario[day] != 'Laboratorio abierto')
    df_horario.loc[mask, day] = 'Clases practicas'

df_horario.head()

,Hora,Lunes,Martes,Miércoles,Jueves,Viernes
0,7:00-7:30,NaN,NaN,NaN,NaN,NaN
1,7:30-8:00,Clases practicas,Laboratorio abierto,Laboratorio abierto,NaN,NaN
2,8:00-8:30,Clases practicas,Laboratorio abierto,Laboratorio abierto,NaN,NaN
3,8:30-9:00,Clases practicas,Laboratorio abierto,Laboratorio abierto,NaN,NaN
4,9:00-9:30,Clases practicas,Clases practicas,Laboratorio abierto,Laboratorio abierto,Clases practicas


In [40]:
df_horario[['Inicio', 'Fin']] = df_horario['Hora'].str.split('-', expand=True)
df_horario = df_horario.drop(columns=['Hora'])

print("df_horario with 'Inicio' and 'Fin':")
display(df_horario)

df_horario with 'Inicio' and 'Fin':


,Lunes,Martes,Miércoles,Jueves,Viernes,Inicio,Fin
0,NaN,NaN,NaN,NaN,NaN,7:00,7:30
1,Clases practicas,Laboratorio abierto,Laboratorio abierto,NaN,NaN,7:30,8:00
2,Clases practicas,Laboratorio abierto,Laboratorio abierto,NaN,NaN,8:00,8:30
3,Clases practicas,Laboratorio abierto,Laboratorio abierto,NaN,NaN,8:30,9:00
4,Clases practicas,Clases practicas,Laboratorio abierto,Laboratorio abierto,Clases practicas,9:00,9:30
5,Clases practicas,Clases practicas,Clases practicas,Laboratorio abierto,Clases practicas,9:30,10:00
6,Clases practicas,Clases practicas,Clases practicas,Laboratorio abierto,Clases practicas,10:00,10:30
7,Clases practicas,Clases practicas,Clases practicas,Laboratorio abierto,Clases practicas,10:30,11:00
8,Clases practicas,Clases practicas,Clases practicas,Laboratorio abierto,Clases practicas,11:00,11:30
9,Clases practicas,Clases practicas,Clases practicas,Clases practicas,Clases practicas,11:30,12:00


In [41]:
file_path_estacion = '/content/Deep_Q_learning/data/df_estacion.csv'
file_path_shelly_iz = '/content/Deep_Q_learning/data/df_shelly_iz.csv'
file_path_temp = '/content/Deep_Q_learning/data/df_temp.csv'

df_estacion = pd.read_csv(file_path_estacion)
df_shelly_iz = pd.read_csv(file_path_shelly_iz)
df_temp = pd.read_csv(file_path_temp)

print("df_estacion head:")
display(df_estacion.head())
print("\ndf_shelly_iz head:")
display(df_shelly_iz.head())
print("\ndf_temp head:")
display(df_temp.head())

df_estacion head:


,sensedAt,temp,Fecha,Hora
0,2025-06-07 00:01:49,24.278847,2025-06-07,00:01:49
1,2025-06-07 00:04:57,24.278847,2025-06-07,00:04:57
2,2025-06-07 00:11:21,24.311024,2025-06-07,00:11:21
3,2025-06-07 00:14:30,24.300299,2025-06-07,00:14:30
4,2025-06-07 00:17:39,24.300299,2025-06-07,00:17:39



df_shelly_iz head:


,sensedAt,potencia_A,Fecha,Hora
0,2025-06-07,0.0,2025-06-07,00:00:00
1,2025-06-07 00:00:10,0.0,2025-06-07,00:00:10
2,2025-06-07 00:00:20,0.0,2025-06-07,00:00:20
3,2025-06-07 00:00:30,0.0,2025-06-07,00:00:30
4,2025-06-07 00:00:40,0.0,2025-06-07,00:00:40



df_temp head:


,sensedAt,temp,Fecha,Hora
0,2025-06-07 00:00:27,25.0,2025-06-07,00:00:27
1,2025-06-07 00:00:54,28.0,2025-06-07,00:00:54
2,2025-06-07 00:01:11,25.0,2025-06-07,00:01:11
3,2025-06-07 00:01:17,25.0,2025-06-07,00:01:17
4,2025-06-07 00:02:40,28.0,2025-06-07,00:02:40


In [42]:
list_sensedAt= pd.concat([df_estacion['sensedAt'],df_shelly_iz['sensedAt'],df_temp['sensedAt']]).unique()
np.sort(list_sensedAt)

array(['2025-06-07', '2025-06-07 00:00:10', '2025-06-07 00:00:20', ...,
       '2025-08-14 23:59:40', '2025-08-14 23:59:44',
       '2025-08-14 23:59:50'], dtype=object)

In [43]:
import pandas as pd
for df in [df_estacion, df_shelly_iz, df_temp]:
    df['sensedAt'] = pd.to_datetime(df['sensedAt'], format='mixed')
all_sensedAt = (
    pd.concat([
        df_estacion['sensedAt'],
        df_shelly_iz['sensedAt'],
        df_temp['sensedAt']
    ])
    .drop_duplicates()
    .sort_values()
    .reset_index(drop=True)
)

df_features = (
    pd.DataFrame({'sensedAt': sorted(all_sensedAt)})
    .reset_index(drop=True)
)
df_features = df_features.merge(
    df_estacion[['sensedAt', 'temp']]
        .rename(columns={'temp': 'temp_externa'}),
    on='sensedAt',
    how='left'
)
df_features = df_features.merge(
    df_temp[['sensedAt', 'temp']]
        .rename(columns={'temp': 'temp_interna'}),
    on='sensedAt',
    how='left'
)
df_features = df_features.merge(
    df_shelly_iz[['sensedAt', 'potencia_A']],
    on='sensedAt',
    how='left'
)
df_features['Fecha'] = df_features['sensedAt'].dt.date
df_features['Hora']  = df_features['sensedAt'].dt.time
df_features = df_features[
    ['sensedAt', 'Fecha', 'Hora',
     'temp_externa', 'temp_interna', 'potencia_A']
]

In [44]:
df_features['estado del aire'] = (df_features['potencia_A'] > 20).astype(int)

In [45]:
import pandas as pd

# ==============================
# CONFIGURACIÓN
# ==============================
# minutos hacia el futuro (ej. 30)
tiempo = 30

# ==============================
# PREPARAR df_horario
# ==============================
def time_to_minutes(t):
    h, m = map(int, t.split(':'))
    return h * 60 + m

df_horario = df_horario.copy()
df_horario['inicio_min'] = df_horario['Inicio'].apply(time_to_minutes)
df_horario['fin_min'] = df_horario['Fin'].apply(time_to_minutes)

# ==============================
# PREPARAR df_features
# ==============================
df_features = df_features.copy()

df_features['datetime'] = pd.to_datetime(df_features['sensedAt'])
df_features['minutes'] = (
    df_features['datetime'].dt.hour * 60 +
    df_features['datetime'].dt.minute
)
df_features['future_minutes'] = df_features['minutes'] + tiempo

day_map = {
    0: 'Lunes',
    1: 'Martes',
    2: 'Miércoles',
    3: 'Jueves',
    4: 'Viernes'
}

df_features['day_col'] = df_features['datetime'].dt.weekday.map(day_map)

# ==============================
# MAPEO DE ACTIVIDADES
# ==============================
def map_activity(val):
    if pd.isna(val):
        return 0
    if val == 'Laboratorio abierto':
        return 1
    if val == 'Clases practicas':
        return 2
    return 0

# ==============================
# CÁLCULO PRINCIPAL
# ==============================
df_features['clases_a_continuacion'] = 0

for _, hrow in df_horario.iterrows():
    start = hrow['inicio_min']
    end = hrow['fin_min']

    overlap = (                                               ###Detectar solapamiento temporal
        (df_features['minutes'] < end) &                      ###Esto detecta si la ventana futura intersecta una clase.
        (df_features['future_minutes'] > start)               ###Ejemplo: ahora = 09:40, futuro = 10:10, clase = 09:00–10:00 -> Sí hay solapamiento
    )

    for day in ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']:
        mask = overlap & (df_features['day_col'] == day)
        clase = map_activity(hrow[day])

        df_features.loc[mask, 'clases_a_continuacion'] = (
            df_features.loc[mask, 'clases_a_continuacion']
            .clip(lower=clase)
        )

# ==============================
# LIMPIEZA
# ==============================
df_features.drop(
    columns=['datetime', 'minutes', 'future_minutes', 'day_col'],
    inplace=True
)

# ==============================
# RESULTADO
# ==============================
print(df_features[['sensedAt', 'clases_a_continuacion']].head())


             sensedAt  clases_a_continuacion
0 2025-06-07 00:00:00                      0
1 2025-06-07 00:00:10                      0
2 2025-06-07 00:00:20                      0
3 2025-06-07 00:00:27                      0
4 2025-06-07 00:00:30                      0


#Fin de carga de datos sensados

#Matcheando datos de cantidad de personas, ubicación y opinión térmica

##Simulación de número de personas

In [46]:
def actividad_actual(row, df_horario):
    dia = row['day_col']

    # Fin de semana o día no mapeado
    if pd.isna(dia):
        return np.nan

    minuto = row['minutes']

    bloque = df_horario[
        (df_horario['inicio_min'] <= minuto) &
        (df_horario['fin_min'] > minuto)
    ]

    if bloque.empty:
        return np.nan

    # Puede haber NaN en el horario (y eso es válido)
    return bloque.iloc[0][dia]


In [47]:
def personas_objetivo(actividad):
    if actividad == 'Clases practicas':
        return np.random.randint(8, 16)

    if actividad == 'Laboratorio abierto':
        return np.random.randint(0, 6)

    # NaN / sin actividad
    return np.random.choice(
        [0, 1, 2, 3, 4],
        p=[0.80, 0.10, 0.06, 0.03, 0.01]
    )


In [48]:
def transicion_suave(actual, objetivo, max_delta=3):
    if actual < objetivo:
        return min(actual + np.random.randint(1, max_delta + 1), objetivo)
    if actual > objetivo:
        return max(actual - np.random.randint(1, max_delta + 1), objetivo)
    return actual


In [49]:
# ==============================
# RECREAR VARIABLES NECESARIAS
# ==============================
df_features = df_features.copy()

df_features['datetime'] = pd.to_datetime(df_features['sensedAt'])
df_features['minutes'] = (
    df_features['datetime'].dt.hour * 60 +
    df_features['datetime'].dt.minute
)

df_features['day_col'] = df_features['datetime'].dt.weekday.map({
    0: 'Lunes',
    1: 'Martes',
    2: 'Miércoles',
    3: 'Jueves',
    4: 'Viernes'
})


n_personas = np.zeros(len(df_features), dtype=int)
n_personas[0] = 0

for i in range(1, len(df_features)):
    actividad = actividad_actual(df_features.loc[i], df_horario)
    objetivo = personas_objetivo(actividad)

    n_personas[i] = transicion_suave(
        actual=n_personas[i - 1],
        objetivo=objetivo,
        max_delta=3
    )

df_features['n_personas'] = n_personas

df_features = df_features.drop(columns=['datetime', 'minutes', 'day_col'])

In [50]:
df_features.head()

,sensedAt,Fecha,Hora,temp_externa,temp_interna,potencia_A,estado del aire,clases_a_continuacion,n_personas
0,2025-06-07 00:00:00,2025-06-07,00:00:00,NaN,NaN,0.0,0,0,0
1,2025-06-07 00:00:10,2025-06-07,00:00:10,NaN,NaN,0.0,0,0,0
2,2025-06-07 00:00:20,2025-06-07,00:00:20,NaN,NaN,0.0,0,0,1
3,2025-06-07 00:00:27,2025-06-07,00:00:27,NaN,25.0,NaN,0,0,1
4,2025-06-07 00:00:30,2025-06-07,00:00:30,NaN,NaN,0.0,0,0,0


##Fin de la simulación de número de personas

##simulación de ubicación de personas

In [51]:
import os

directory_path = '/content/Deep_Q_learning/data/analisis en R_data_personas/Personas'

all_files_and_dirs = os.listdir(directory_path)

csv_files = [f for f in all_files_and_dirs if f.endswith('.csv')]

csv_files_sorted = sorted(csv_files)

# Initialize an empty dictionary to store DataFrames
dataframes_dict = {}

print(f"CSV files in '{directory_path}' (orden ascendente):")
for csv_file in csv_files_sorted:
    print(csv_file)

    # Construct the full path to the CSV file
    full_file_path = os.path.join(directory_path, csv_file)

    # Generate a dynamic DataFrame name (e.g., df_523008insert)
    df_name = 'df_' + os.path.splitext(csv_file)[0]

    # Load the CSV file into a DataFrame and store it in the dictionary
    dataframes_dict[df_name] = pd.read_csv(full_file_path)

    print(f"Loaded {csv_file} into DataFrame: {df_name}")

print("\nAll CSV files loaded into DataFrames in 'dataframes_dict'.")

CSV files in '/content/Deep_Q_learning/data/analisis en R_data_personas/Personas' (orden ascendente):
523001insert.csv
Loaded 523001insert.csv into DataFrame: df_523001insert
523002insert.csv
Loaded 523002insert.csv into DataFrame: df_523002insert
523003insert.csv
Loaded 523003insert.csv into DataFrame: df_523003insert
523004insert.csv
Loaded 523004insert.csv into DataFrame: df_523004insert
523005insert.csv
Loaded 523005insert.csv into DataFrame: df_523005insert
523006insert.csv
Loaded 523006insert.csv into DataFrame: df_523006insert
523007insert.csv
Loaded 523007insert.csv into DataFrame: df_523007insert
523008insert.csv
Loaded 523008insert.csv into DataFrame: df_523008insert
523009insert.csv
Loaded 523009insert.csv into DataFrame: df_523009insert
523010insert.csv
Loaded 523010insert.csv into DataFrame: df_523010insert
523011insert.csv
Loaded 523011insert.csv into DataFrame: df_523011insert
523012insert.csv
Loaded 523012insert.csv into DataFrame: df_523012insert
523013insert.csv
Loade

In [52]:
# =====================================================
# ASIGNACIÓN SECUENCIAL DE RUTAS A df_features
# USANDO dataframes_dict (YA CARGADO)
# =====================================================

import random
import pandas as pd

# -----------------------------------------------------
# 1. PREPARAR RUTAS (una por df_5230**insert)
# -----------------------------------------------------
# dataframes_dict ya existe y contiene:
# { 'df_523001insert': df, 'df_523002insert': df, ... }

rutas = {}

for key, df_ruta in dataframes_dict.items():
    df_ruta = df_ruta.copy()
    df_ruta['Time'] = pd.to_datetime(df_ruta['Time'])
    rutas[key] = df_ruta

# -----------------------------------------------------
# 2. PREPARAR df_features
# -----------------------------------------------------
df = df_features.copy()
df['sensedAt'] = pd.to_datetime(df['sensedAt'])

# columna ubicacion: lista de duplas (x, y)
df['ubicacion'] = [[] for _ in range(len(df))]

# -----------------------------------------------------
# 3. POOL DE RUTAS SIN REPETICIÓN
# -----------------------------------------------------
rutas_ids = list(rutas.keys())
random.shuffle(rutas_ids)

rutas_disponibles = rutas_ids.copy()
rutas_usadas = []

def tomar_ruta():
    global rutas_disponibles, rutas_usadas
    if len(rutas_disponibles) == 0:
        rutas_disponibles = rutas_usadas.copy()
        rutas_usadas = []
        random.shuffle(rutas_disponibles)
    r = rutas_disponibles.pop(0)
    rutas_usadas.append(r)
    return r

# -----------------------------------------------------
# 4. ALGORITMO PRINCIPAL (TU PSEUDOCÓDIGO)
# -----------------------------------------------------
max_personas = int(df['n_personas'].max())

for k in range(1, max_personas + 1):

    i = 0
    while i < len(df):

        # buscar inicio de una nueva persona virtual
        if df.loc[i, 'n_personas'] >= k:

            ruta_id = tomar_ruta()
            ruta = rutas[ruta_id]
            j = 0  # índice dentro de la ruta

            # avanzar mientras la persona "exista"
            while (
                i < len(df) and
                j < len(ruta) and
                df.loc[i, 'n_personas'] >= k
            ):
                x = ruta.iloc[j]['world_x']
                y = ruta.iloc[j]['world_y']

                df.at[i, 'ubicacion'].append((x, y))

                i += 1
                j += 1
        else:
            i += 1

# -----------------------------------------------------
# 5. VERIFICACIÓN DE CONSISTENCIA
# -----------------------------------------------------
df['check'] = df['ubicacion'].apply(len) == df['n_personas']
print(df['check'].value_counts())

# df es el df_features final con ubicaciones asignadas


check
True    121968
Name: count, dtype: int64


In [53]:
df_features['ubicacion'] = df['ubicacion']

In [54]:
import numpy as np
import pandas as pd

# ==============================
# PARÁMETROS
# ==============================
DISP_TH = 3.0  # umbral de dispersión (ajústalo si es necesario)

# Rectángulos (x_min, x_max, y_min, y_max)
RECT_1 = (0.0, 18.0, -7.5, 1.0)
RECT_2 = (0.0, 18.0, -17.5, -7.5)

# ==============================
# FUNCIÓN PRINCIPAL
# ==============================
def clasificar_ubicacion(lista_puntos):
    if not lista_puntos or len(lista_puntos) == 0:
        return 0

    xs = np.array([p[0] for p in lista_puntos], dtype=float)
    ys = np.array([p[1] for p in lista_puntos], dtype=float)

    mean_x, mean_y = xs.mean(), ys.mean()
    std_x, std_y = xs.std(), ys.std()

    # Dispersión alta
    if std_x >= DISP_TH or std_y >= DISP_TH:
        return 0

    # Dispersión baja → verificar rectángulos
    if (RECT_1[0] <= mean_x <= RECT_1[1]) and (RECT_1[2] <= mean_y <= RECT_1[3]):
        return 1

    if (RECT_2[0] <= mean_x <= RECT_2[1]) and (RECT_2[2] <= mean_y <= RECT_2[3]):
        return 2

    return 0

# ==============================
# APLICAR AL DATAFRAME
# ==============================
df_features['ubicacion_relativa'] = df_features['ubicacion'].apply(clasificar_ubicacion)


In [55]:
import numpy as np

df_features['ubicacion'] = df_features['ubicacion'].apply(lambda x: np.nan if len(x) == 0 else x)
df_features.loc[df_features['ubicacion'].isna(), 'ubicacion_relativa'] = np.nan

## fin de simulación de ubicación de personas

#Añadir datos de opinión

| Variable                         |  Rango discretizado (niveles)                         | N niveles |
| :------------------------------- |:--------------------------------------------------- | :-------- |
| Panel de opinión térmica       | 0: muy fría, 1: neutra, 2: muy calurosa | 3         |

In [56]:
import numpy as np

# Inicializar la columna como NaN
df_features['thermal_opinion'] = np.nan

# Máscara: solo filas con personas
mask_personas = df_features['n_personas'] > 0

# Asignar valores aleatorios solo a esas filas
df_features.loc[mask_personas, 'thermal_opinion'] = np.random.choice(
    [0, 1, 2],
    size=mask_personas.sum(),
    p=[0.2, 0.6, 0.2]
)

#archivo de df_features

In [57]:
output_file_path = 'df_features.csv'
df_features.to_csv(output_file_path, index=False)
print(f"DataFrame 'df_features' exported successfully to '{output_file_path}'")

DataFrame 'df_features' exported successfully to 'df_features.csv'


In [58]:
df_features.head()

,sensedAt,Fecha,Hora,temp_externa,temp_interna,potencia_A,estado del aire,clases_a_continuacion,n_personas,ubicacion,ubicacion_relativa,thermal_opinion
0,2025-06-07 00:00:00,2025-06-07,00:00:00,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN
1,2025-06-07 00:00:10,2025-06-07,00:00:10,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN
2,2025-06-07 00:00:20,2025-06-07,00:00:20,NaN,NaN,0.0,0,0,1,"[(10.722, -2.27)]",1.0,1.0
3,2025-06-07 00:00:27,2025-06-07,00:00:27,NaN,25.0,NaN,0,0,1,"[(10.708, -2.32)]",1.0,1.0
4,2025-06-07 00:00:30,2025-06-07,00:00:30,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN


In [59]:
df_features[95991:96000]

,sensedAt,Fecha,Hora,temp_externa,temp_interna,potencia_A,estado del aire,clases_a_continuacion,n_personas,ubicacion,ubicacion_relativa,thermal_opinion
95991,2025-08-12 15:49:40,2025-08-12,15:49:40,NaN,NaN,-1.55,0,1,5,"[(3.77, -3.763), (4.007, -4.251), (5.067, -8.9...",1.0,1.0
95992,2025-08-12 15:49:50,2025-08-12,15:49:50,NaN,NaN,0.00,0,1,4,"[(3.77, -3.766), (4.05, -4.227), (5.087, -8.92...",1.0,1.0
95993,2025-08-12 15:50:00,2025-08-12,15:50:00,NaN,NaN,-1.50,0,1,2,"[(3.77, -3.768), (4.112, -4.244)]",1.0,1.0
95994,2025-08-12 15:50:09,2025-08-12,15:50:09,NaN,26.0,NaN,0,1,1,"[(3.77, -3.771)]",1.0,1.0
95995,2025-08-12 15:50:10,2025-08-12,15:50:10,NaN,NaN,-1.55,0,1,4,"[(3.77, -3.774), (10.722, -2.27), (3.984, -4.2...",1.0,1.0
95996,2025-08-12 15:50:20,2025-08-12,15:50:20,NaN,NaN,-1.57,0,1,2,"[(3.77, -3.776), (10.708, -2.32)]",0.0,1.0
95997,2025-08-12 15:50:30,2025-08-12,15:50:30,NaN,NaN,-1.53,0,1,2,"[(3.77, -3.779), (12.017, -2.047)]",0.0,1.0
95998,2025-08-12 15:50:40,2025-08-12,15:50:40,NaN,NaN,0.00,0,1,1,"[(3.77, -3.781)]",1.0,1.0
95999,2025-08-12 15:50:47,2025-08-12,15:50:47,NaN,27.0,NaN,0,1,1,"[(3.77, -3.784)]",1.0,1.0
